In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2022/train.csv
/kaggle/input/tabular-playground-series-feb-2022/test.csv


In [4]:
import torch

if torch.cuda.is_available():
    print('GPU is available.')
else:
    print('GPU is not available.')


GPU is available.


In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
train.head()


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [8]:
print(train.columns)

Index(['row_id', 'A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6',
       'A0T0G5C5', 'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2',
       ...
       'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0', 'A9T0G0C1',
       'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0', 'target'],
      dtype='object', length=288)


In [28]:
print('Train Shape: {}\nMissing Data: {}\nDuplicates: {}\n'\
      .format(train.shape, train.isna().sum().sum(), train.duplicated().sum()))

Train Shape: (200000, 288)
Missing Data: 0
Duplicates: 0



In [10]:
# Extract the target column (assuming the column is named 'target' in this example)
target_column = train['target']

In [11]:
train=train.drop(columns=['target']) 

In [13]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Example data (replace this with your actual data)
train = np.array(train)

# Function to perform Min-Max scaling
def min_max_scaling(train):
    # Create MinMaxScaler object
    scaler = MinMaxScaler()
    
    # Fit the scaler to the data and transform the data
    scaled_data = scaler.fit_transform(train)
    
    # Get the minimum and maximum values for each feature
    min_vals = scaler.data_min_
    max_vals = scaler.data_max_
    
    return scaled_data, min_vals, max_vals

# Apply Min-Max scaling
scaled_data, min_vals, max_vals = min_max_scaling(train)

# Print the scaled data, minimum, and maximum values for each feature
print("\nScaled Data (Min-Max scaled between 0 and 1):\n", scaled_data)
print("\nMin Values for each feature:\n", min_vals)
print("\nMax Values for each feature:\n", max_vals)



Scaled Data (Min-Max scaled between 0 and 1):
 [[0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [5.000025e-06 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000005e-05 0.000000e+00 8.000000e-04 ... 1.800000e-03 1.450000e-03
  2.000000e-03]
 ...
 [9.999900e-01 1.000000e-04 1.100000e-03 ... 4.500000e-03 1.550000e-03
  1.000000e-03]
 [9.999950e-01 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000000e+00 2.000000e-04 8.000000e-04 ... 2.300000e-03 2.150000e-03
  0.000000e+00]]

Min Values for each feature:
 [ 0.00000000e+00 -9.53674316e-07 -9.53674316e-06 -4.29153442e-05
 -1.14440918e-04 -2.00271606e-04 -2.40325928e-04 -2.00271606e-04
 -1.14440918e-04 -4.29153442e-05 -9.53674316e-06 -9.53674316e-07
 -9.53674316e-06 -8.58306885e-05 -3.43322754e-04 -8.01086426e-04
 -1.20162964e-03 -1.20162964e-03 -8.01086426e-04 -3.43322754e-04
 -8.58306885e-05 -9.53674316e-06 -4.29153442e-05 -3.43322754e-04
 -1

In [14]:
import numpy as np

from sklearn.decomposition import PCA
# This line imports the PCA class from the scikit-learn library, which is used for dimensionality reduction 
# using Principal Component Analysis.

# Assuming X_train_scaled is your scaled training data
# You can use X_train_formatted if you've converted it to human-readable format X_train_formatted

# Initialize the PCA model with the desired number of components
n_components = 33  # Adjust this value as needed, desired number of component.

pca = PCA(n_components=n_components)
# This initializes a PCA model with the specified number of components.

# Fit the PCA model to your scaled data
pca.fit(scaled_data)   
# This fits the PCA model to the scaled data

# Transform the data to its principal components
principal_compo = pca.transform(scaled_data)
# This line transforms the scaled data into its principal components using the trained PCA model.

# Create a DataFrame for the PCA results using common columns
pca_df = pd.DataFrame(data=principal_compo, columns=[f'PC{i}' for i in range(1, 34)])
# This creates a DataFrame (pca_df) to store the principal components. Each column is labeled with a prefix 'PC' followed 
# by the component number.
# Printing the PCA results
print(pca_df)

# Now, X_train_pca contains your training data reduced to the specified number of principal components
# You can use X_train_pca for classification

             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -0.195312  1.450241 -0.493021 -0.572658 -0.297634  0.197065  0.409359   
1      -0.455772  0.023449 -0.499954 -0.147321  0.226932  0.058239 -0.082557   
2      -0.455364 -0.176181 -0.501369  0.009695 -0.088106 -0.045754 -0.005821   
3      -0.514005 -0.202394 -0.502223  0.088732 -0.101302  0.066162 -0.007537   
4       0.498484  1.020506 -0.501142  0.310238 -0.013760 -0.050889 -0.443492   
...          ...       ...       ...       ...       ...       ...       ...   
199995 -0.481202 -0.183816  0.498148  0.051616 -0.089788 -0.002522 -0.010078   
199996 -0.196881 -0.019007  0.500695 -0.205748  0.337806 -0.063222 -0.172514   
199997 -0.212165 -0.158487  0.500078 -0.081012 -0.065431 -0.176711 -0.001364   
199998 -0.364806  0.009943  0.501534 -0.485663  0.894588  0.268238  0.036826   
199999 -0.871026 -0.253236  0.495066  0.280512 -0.152345  0.362412 -0.034244   

             PC8       PC9      PC10  .

In [15]:
pca_df['target'] = target_column

In [16]:
y = pca_df['target']
X = pca_df.drop(columns=['target'])

In [17]:
import pandas as pd

In [18]:
# Convert y to a pandas Series
y = pd.Series(y)
# Convert X to a pandas DataFrame
X = pd.DataFrame(X)

In [19]:
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [20]:
# Ensure the target variable is categorical
y = pd.Categorical(y)
y_encoded = y.codes  # Convert to integer codes
num_classes = len(y.categories)

In [21]:
# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [23]:
import numpy as np
import pandas as pd
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.initializers import he_normal

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record

# Define the multi-layered dense neural network model
def create_dense_nn(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(512, activation='relu', kernel_initializer=he_normal(), input_shape=input_shape, kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(64, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(512, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(64, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(512, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(64, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adamax(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

for train_index, test_index in sss.split(X, y_encoded):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    
    print("Before KMeans")
    
    # Apply KMeans clustering on the training set using cuML
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array
    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values  # Convert to NumPy array
    y_train_reduced = reduced_train_data['target'].values  # Convert to NumPy array
    
    # One-hot encode the labels
    y_train_reduced_categorical = to_categorical(y_train_reduced, num_classes=num_classes)
    
    # Create the dense neural network model
    cnn_model = create_dense_nn(input_shape=(X_train_reduced.shape[1],), num_classes=num_classes)
    
    # Train the model on the reduced training set
    cnn_model.fit(X_train_reduced, y_train_reduced_categorical, epochs=100, batch_size=32, verbose=0)  # Set verbose=0
    
    # Predict on the training set
    y_train_pred = cnn_model.predict(X_train_reduced)
    y_train_pred_classes = np.argmax(y_train_pred, axis=1)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred_classes)
    train_accuracies.append(train_accuracy)
    
    # One-hot encode the test labels
    y_test_categorical = to_categorical(y_test, num_classes=num_classes)
    
    # Predict on the original test set
    y_test_pred = cnn_model.predict(X_test.values)
    y_test_pred_classes = np.argmax(y_test_pred, axis=1)
    test_accuracy = accuracy_score(y_test, y_test_pred_classes)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy: {train_accuracy:.4f}')
    print(f'Fold test accuracy: {test_accuracy:.4f}')
#     break

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy: {average_train_accuracy:.4f}')
print(f'Average test accuracy: {average_test_accuracy:.4f}')

Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9889
Fold test accuracy: 0.8534
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9856
Fold test accuracy: 0.8404
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9883
Fold test accuracy: 0.8410
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9875
Fold test accuracy: 0.8508
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9809
Fold test accuracy: 0.8343
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9809
Fold test accuracy: 0.8537
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9811
Fold test accuracy: 0.8440
Before KMeans
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9820
Fold test a

In [15]:
from cuml.cluster import KMeans as cuKMeans  # Import cuML's KMeans for GPU acceleration
from sklearn.ensemble import AdaBoostClassifier  # Import AdaBoost for classification
from sklearn.model_selection import StratifiedShuffleSplit  # Import StratifiedShuffleSplit for cross-validation
from sklearn.metrics import accuracy_score  # Import accuracy_score for evaluation
import numpy as np  # Import NumPy for numerical operations


In [16]:
from sklearn.tree import DecisionTreeClassifier

base_estimator = DecisionTreeClassifier(max_depth=5)  # You can adjust max_depth
# Step 4: Initialize the AdaBoost Classifier
model_4 = AdaBoostClassifier(estimator=base_estimator,n_estimators=50,learning_rate=1)
# Initialize the AdaBoost model
# model_4 = AdaBoostClassifier()


In [17]:
# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [19]:
print("AdaBoost  Results:-")

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print(f'Before kmean:{X_train.shape}')
    # Apply KMeans clustering on the training set using cuML
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array
    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
#     print(f'After kmean:{X_train_reduced.shape}')

    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values  # Convert to NumPy array
    y_train_reduced = reduced_train_data['target'].values  # Convert to NumPy array
    
#     print("After KMeans")
#     print(X_train_reduced.shape)
    
    # Train the model on the reduced training set
    model_4.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_4.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_4.predict(X_test.values)  # Convert to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for adaboost: {train_accuracy:.4f}')
    print(f'Fold test accuracy for adaboost: {test_accuracy:.4f}')
    print("-------------------------------------")
#     break
    
# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for adaboost: {average_train_accuracy:.4f}')
print(f'Average test accuracy for adaboost: {average_test_accuracy:.4f}')

AdaBoost  Results:-
Fold training accuracy for adaboost: 0.5762
Fold test accuracy for adaboost: 0.5152
-------------------------------------
Fold training accuracy for adaboost: 0.5932
Fold test accuracy for adaboost: 0.5361
-------------------------------------
Fold training accuracy for adaboost: 0.5887
Fold test accuracy for adaboost: 0.5172
-------------------------------------
Fold training accuracy for adaboost: 0.5791
Fold test accuracy for adaboost: 0.5452
-------------------------------------
Fold training accuracy for adaboost: 0.6015
Fold test accuracy for adaboost: 0.5494
-------------------------------------
Fold training accuracy for adaboost: 0.5963
Fold test accuracy for adaboost: 0.5608
-------------------------------------
Fold training accuracy for adaboost: 0.5985
Fold test accuracy for adaboost: 0.5143
-------------------------------------
Fold training accuracy for adaboost: 0.5701
Fold test accuracy for adaboost: 0.5450
-------------------------------------
Fold

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

In [22]:
# Initialize the Naive Bayes classifier
model_3 = GaussianNB()

In [23]:
# from sklearn.model_selection import StratifiedShuffleSplit

# Define the StratifiedShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)

# Lists to store results
train_accuracies = []
test_accuracies = []

In [25]:
print("Naive bayes Result")
# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print("Before KMeans")
#     print(X_train.shape)
    
    # Apply KMeans clustering on the training set using cuML
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array
    
    # Create a DataFrame with cluster labels
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values  # Convert to NumPy array
    y_train_reduced = reduced_train_data['target'].values  # Convert to NumPy array
    
#     print("After KMeans")
#     print(X_train_reduced.shape)
    
    # Train the model on the reduced training set
    model_3.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_3.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_3.predict(X_test.values)  # Convert to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for Naive : {train_accuracy:.2f}')
    print(f'Fold test accuracy for Naive: {test_accuracy:.2f}')
    print("---------------------------")
#     break

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy Naive: {average_train_accuracy:.4f}')
print(f'Average test accuracy for Naive: {average_test_accuracy:.4f}')


Naive bayes Result
Fold training accuracy for Naive : 0.52
Fold test accuracy for Naive: 0.46
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.44
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.43
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.47
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.44
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.45
---------------------------
Fold training accuracy for Naive : 0.52
Fold test accuracy for Naive: 0.46
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.45
---------------------------
Fold training accuracy for Naive : 0.52
Fold test accuracy for Naive: 0.46
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accu

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import cudf
import cuml

from sklearn.svm import SVC

In [21]:
# Define the model
model_2 = SVC(kernel='linear', C=1.0)

In [22]:
from sklearn.model_selection import StratifiedShuffleSplit

# Define the StratifiedShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)

# Lists to store results
train_accuracies = []
test_accuracies = []

In [23]:
print("SVM Results")
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration
# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print("Before Kmean")
#     print(X_train.shape)
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array

    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
#     print("After kmean")
#     print(X_train_reduced.shape)
    # Train the model on the reduced training set
    model_2.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_2.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_2.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold model_2 training  accuracy for SVM: {train_accuracy:.4f}')
    print(f'Fold model_2 test accuracy for SVM: {test_accuracy:.4f}')
    print("---------------------------")
#     break
    

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for SVM: {average_train_accuracy:.4f}')
print(f'Average test accuracy for SVM: {average_test_accuracy:.4f}')

SVM Results
Fold model_2 training  accuracy for SVM: 0.6371
Fold model_2 test accuracy for SVM: 0.5920
---------------------------
Fold model_2 training  accuracy for SVM: 0.6405
Fold model_2 test accuracy for SVM: 0.5859
---------------------------
Fold model_2 training  accuracy for SVM: 0.6339
Fold model_2 test accuracy for SVM: 0.5977
---------------------------
Fold model_2 training  accuracy for SVM: 0.6395
Fold model_2 test accuracy for SVM: 0.5999
---------------------------
Fold model_2 training  accuracy for SVM: 0.6372
Fold model_2 test accuracy for SVM: 0.5955
---------------------------
Fold model_2 training  accuracy for SVM: 0.6367
Fold model_2 test accuracy for SVM: 0.6055
---------------------------
Fold model_2 training  accuracy for SVM: 0.6428
Fold model_2 test accuracy for SVM: 0.6085
---------------------------
Fold model_2 training  accuracy for SVM: 0.6343
Fold model_2 test accuracy for SVM: 0.5857
---------------------------
Fold model_2 training  accuracy for 

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import cudf
import cuml

In [25]:
from sklearn.model_selection import StratifiedShuffleSplit

# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [27]:
model_1 = RandomForestClassifier(class_weight="balanced",n_estimators=1000, max_depth = 9, min_samples_split=10,random_state=42)

In [29]:

# from cuml.ensemble import RandomForestClassifier as cuRFClassifier  # Import cuML RandomForest for GPU acceleration
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array

    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
    # Train the model on the reduced training set
    model_1.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_1.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_1.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for Random Forest : {train_accuracy:.4f}')
    print(f'Fold test accuracy for Random Forest: {test_accuracy:.4f}')
    print("------------------------------")
#     break
    
# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for RF: {average_train_accuracy:.4f}')
print(f'Average test accuracy for RF: {average_test_accuracy:.4f}')

Fold training accuracy for Random Forest : 0.9049
Fold test accuracy for Random Forest: 0.7617
------------------------------
Fold training accuracy for Random Forest : 0.9106
Fold test accuracy for Random Forest: 0.7678
------------------------------
Fold training accuracy for Random Forest : 0.9047
Fold test accuracy for Random Forest: 0.7633
------------------------------
Fold training accuracy for Random Forest : 0.9088
Fold test accuracy for Random Forest: 0.7584
------------------------------
Fold training accuracy for Random Forest : 0.9045
Fold test accuracy for Random Forest: 0.7728
------------------------------
Fold training accuracy for Random Forest : 0.9056
Fold test accuracy for Random Forest: 0.7511
------------------------------
Fold training accuracy for Random Forest : 0.9093
Fold test accuracy for Random Forest: 0.7722
------------------------------
Fold training accuracy for Random Forest : 0.9069
Fold test accuracy for Random Forest: 0.7612
-----------------------

In [ ]:
-------------END-----------------------

In [35]:


# This function accepts a dataframe.
# It selects the dataframe's numerical columns and normalizes those columns.
# The dataframe with normalized values are returned.
def normalize_numerical_features(train_features, test_features):
    
    train_numerical_features = train_features._get_numeric_data() # Selecting all the numerical features of Train
    test_numerical_features = test_features._get_numeric_data() # Selecting all the numerical features of Test
    numerical_columns = list(set(train_numerical_features.columns).intersection(set(test_numerical_features.columns)))
    train_numerical_features = train_numerical_features[numerical_columns]
    test_numerical_features = test_numerical_features[numerical_columns]
    
    # Determining the value_range and min_value to be used in normalization using Train
    value_range = train_numerical_features.max() - train_numerical_features.min()
    min_value = train_numerical_features.min()
    
    # Getting the normalized feature values
    train_normalized_numerical_features = (train_numerical_features - min_value) / value_range
    test_normalized_numerical_features = (test_numerical_features - min_value) / value_range
    
    # Replacing the normalized feature values in the original dataset.
    train_features[numerical_columns] = train_normalized_numerical_features
    test_features[numerical_columns] = test_normalized_numerical_features
    
 
    # All the numerical features in train and test are now normalized.
    return (train_features, test_features)
train_features, test_features = normalize_numerical_features(train_features, test_features)
print('TRAIN FEATURES:')
train_features
print('\n\nTEST FEATURES:')
test_features

TRAIN FEATURES:


TEST FEATURES:


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
119737,0.598686,0.0000,0.0010,0.0010,0.0130,0.01350,0.01050,0.00950,0.0080,0.0040,...,0.00650,0.01450,0.00800,0.01700,0.01200,0.01100,0.0030,0.0040,0.0025,0.000
72272,0.361359,0.0000,0.0020,0.0050,0.0110,0.00850,0.01300,0.00750,0.0130,0.0000,...,0.00600,0.01300,0.00900,0.01750,0.02050,0.01550,0.0020,0.0030,0.0030,0.000
158154,0.790773,0.0000,0.0003,0.0020,0.0045,0.00440,0.00470,0.00435,0.0048,0.0022,...,0.01045,0.02935,0.01775,0.02955,0.03215,0.01825,0.0074,0.0109,0.0048,0.002
65426,0.327128,0.0001,0.0001,0.0025,0.0157,0.01150,0.01165,0.01010,0.0161,0.0024,...,0.00700,0.01300,0.00710,0.01425,0.01575,0.01055,0.0033,0.0025,0.0028,0.000
30074,0.150367,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.50000,1.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.50000,0.0000,0.0000,0.0000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97771,0.488855,0.0001,0.0011,0.0045,0.0116,0.01165,0.01265,0.01100,0.0132,0.0047,...,0.00330,0.00625,0.00345,0.00635,0.00695,0.00355,0.0013,0.0013,0.0010,0.001
59813,0.299063,0.0000,0.0020,0.0050,0.0110,0.00900,0.01300,0.00750,0.0130,0.0000,...,0.01000,0.02350,0.01500,0.01900,0.02500,0.00950,0.0030,0.0070,0.0030,0.000
103735,0.518675,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000,0.000
180226,0.901134,0.0000,0.0020,0.0050,0.0170,0.01200,0.01650,0.01200,0.0220,0.0010,...,0.00300,0.01050,0.00750,0.01000,0.01250,0.00250,0.0020,0.0040,0.0005,0.000


In [36]:
print('TRAIN FEATURES:')
train_features

TRAIN FEATURES:


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
21269,0.106341,0.0000,0.0020,0.0050,0.0110,0.00850,0.01300,0.00750,0.0130,0.0000,...,0.00800,0.01350,0.00900,0.01600,0.01550,0.00900,0.0030,0.0020,0.00200,0.000
187660,0.938304,0.0001,0.0012,0.0091,0.0462,0.05110,0.06795,0.05280,0.0457,0.0075,...,0.00325,0.00605,0.00380,0.00680,0.00765,0.00425,0.0022,0.0018,0.00140,0.001
774,0.003865,0.0001,0.0013,0.0043,0.0099,0.01035,0.01180,0.00975,0.0117,0.0046,...,0.00305,0.00750,0.00385,0.00760,0.00775,0.00420,0.0022,0.0026,0.00095,0.001
184577,0.922889,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.05000,0.00000,0.00000,0.00000,0.0000,0.0000,0.00000,0.000
37127,0.185632,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.00000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.599396,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.00000,0.000
103694,0.518470,0.0000,0.0010,0.0040,0.0150,0.01550,0.01350,0.01050,0.0150,0.0020,...,0.00600,0.00950,0.00400,0.01100,0.01000,0.00250,0.0010,0.0030,0.00100,0.000
131932,0.659662,0.0003,0.0009,0.0029,0.0058,0.00665,0.00680,0.00575,0.0066,0.0028,...,0.01630,0.03670,0.02165,0.04940,0.06465,0.04640,0.0143,0.0213,0.01185,0.002
146867,0.734337,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.05000,0.05000,0.00000,0.05000,0.10000,0.00000,0.1000,0.1000,0.00000,0.000


In [37]:
print('\n\nTEST FEATURES:')
test_features



TEST FEATURES:


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
119737,0.598686,0.0000,0.0010,0.0010,0.0130,0.01350,0.01050,0.00950,0.0080,0.0040,...,0.00650,0.01450,0.00800,0.01700,0.01200,0.01100,0.0030,0.0040,0.0025,0.000
72272,0.361359,0.0000,0.0020,0.0050,0.0110,0.00850,0.01300,0.00750,0.0130,0.0000,...,0.00600,0.01300,0.00900,0.01750,0.02050,0.01550,0.0020,0.0030,0.0030,0.000
158154,0.790773,0.0000,0.0003,0.0020,0.0045,0.00440,0.00470,0.00435,0.0048,0.0022,...,0.01045,0.02935,0.01775,0.02955,0.03215,0.01825,0.0074,0.0109,0.0048,0.002
65426,0.327128,0.0001,0.0001,0.0025,0.0157,0.01150,0.01165,0.01010,0.0161,0.0024,...,0.00700,0.01300,0.00710,0.01425,0.01575,0.01055,0.0033,0.0025,0.0028,0.000
30074,0.150367,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.50000,1.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.50000,0.0000,0.0000,0.0000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97771,0.488855,0.0001,0.0011,0.0045,0.0116,0.01165,0.01265,0.01100,0.0132,0.0047,...,0.00330,0.00625,0.00345,0.00635,0.00695,0.00355,0.0013,0.0013,0.0010,0.001
59813,0.299063,0.0000,0.0020,0.0050,0.0110,0.00900,0.01300,0.00750,0.0130,0.0000,...,0.01000,0.02350,0.01500,0.01900,0.02500,0.00950,0.0030,0.0070,0.0030,0.000
103735,0.518675,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000,0.000
180226,0.901134,0.0000,0.0020,0.0050,0.0170,0.01200,0.01650,0.01200,0.0220,0.0010,...,0.00300,0.01050,0.00750,0.01000,0.01250,0.00250,0.0020,0.0040,0.0005,0.000


In [ ]:
train_class=y_trainnew
test_class=y_test

In [ ]:
##Converting class labels to numeric values


# This function accepts the class labels of train and test
# It generates a mapper that maps from the class labels to the output neuron index number
# For example if we consider IRIS dataset then the mapper would be like:
# iris setosa -> 0
# iris virginica -> 1
# iris versicolor -> 2
def class_label_to_output_neuron_index_mapper(train_class, test_class):
    
    class_label_to_output_neuron_index_mapper = {j: i for i, j in enumerate(train_class.astype('category').cat.categories)} 
 
    train_class = train_class.map(class_label_to_output_neuron_index_mapper)
    test_class = test_class.map(class_label_to_output_neuron_index_mapper)
    if test_class.dtype != train_class.dtype:
        train_class = train_class.astype(test_class.dtype)
 
    return (class_label_to_output_neuron_index_mapper, train_class, test_class)

In [15]:
print('BEFORE:')
print('\n\nTRAIN CLASS:')
train_class
print('\n\nTEST CLASS')
test_class

the_map, train_class, test_class = class_label_to_output_neuron_index_mapper(train_class, test_class)

print('\n\nAFTER:')
print('\nTHE MAP:')
the_map
print('\n\nTRAIN CLASS:')
train_class
print('\n\nTEST CLASS')
test_class

BEFORE:


TRAIN CLASS:


TEST CLASS


AFTER:

THE MAP:


TRAIN CLASS:


TEST CLASS


119737    4
72272     7
158154    9
65426     4
30074     5
         ..
97771     4
59813     2
103735    4
180226    3
119389    9
Name: target, Length: 60000, dtype: int64

In [16]:
# Finally we concatinate the feature columns and the class label column

encoded_train = pd.concat([train_features, train_class], axis=1)
encoded_test = pd.concat([test_features, test_class], axis=1)


print('\n\nENCODED TEST:')
encoded_test



ENCODED TEST:


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
119737,0.598686,0.0000,0.0010,0.0010,0.0130,0.01350,0.01050,0.00950,0.0080,0.0040,...,0.01450,0.00800,0.01700,0.01200,0.01100,0.0030,0.0040,0.0025,0.000,4
72272,0.361359,0.0000,0.0020,0.0050,0.0110,0.00850,0.01300,0.00750,0.0130,0.0000,...,0.01300,0.00900,0.01750,0.02050,0.01550,0.0020,0.0030,0.0030,0.000,7
158154,0.790773,0.0000,0.0003,0.0020,0.0045,0.00440,0.00470,0.00435,0.0048,0.0022,...,0.02935,0.01775,0.02955,0.03215,0.01825,0.0074,0.0109,0.0048,0.002,9
65426,0.327128,0.0001,0.0001,0.0025,0.0157,0.01150,0.01165,0.01010,0.0161,0.0024,...,0.01300,0.00710,0.01425,0.01575,0.01055,0.0033,0.0025,0.0028,0.000,4
30074,0.150367,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.50000,1.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.50000,0.0000,0.0000,0.0000,0.000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97771,0.488855,0.0001,0.0011,0.0045,0.0116,0.01165,0.01265,0.01100,0.0132,0.0047,...,0.00625,0.00345,0.00635,0.00695,0.00355,0.0013,0.0013,0.0010,0.001,4
59813,0.299063,0.0000,0.0020,0.0050,0.0110,0.00900,0.01300,0.00750,0.0130,0.0000,...,0.02350,0.01500,0.01900,0.02500,0.00950,0.0030,0.0070,0.0030,0.000,2
103735,0.518675,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000,0.000,4
180226,0.901134,0.0000,0.0020,0.0050,0.0170,0.01200,0.01650,0.01200,0.0220,0.0010,...,0.01050,0.00750,0.01000,0.01250,0.00250,0.0020,0.0040,0.0005,0.000,3


In [17]:
print('ENCODED TRAIN:')
encoded_train

ENCODED TRAIN:


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
21269,0.106341,0.0000,0.0020,0.0050,0.0110,0.00850,0.01300,0.00750,0.0130,0.0000,...,0.01350,0.00900,0.01600,0.01550,0.00900,0.0030,0.0020,0.00200,0.000,9
187660,0.938304,0.0001,0.0012,0.0091,0.0462,0.05110,0.06795,0.05280,0.0457,0.0075,...,0.00605,0.00380,0.00680,0.00765,0.00425,0.0022,0.0018,0.00140,0.001,5
774,0.003865,0.0001,0.0013,0.0043,0.0099,0.01035,0.01180,0.00975,0.0117,0.0046,...,0.00750,0.00385,0.00760,0.00775,0.00420,0.0022,0.0026,0.00095,0.001,9
184577,0.922889,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.05000,0.00000,0.00000,0.00000,0.0000,0.0000,0.00000,0.000,4
37127,0.185632,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.00000,0.000,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.599396,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.00000,0.000,8
103694,0.518470,0.0000,0.0010,0.0040,0.0150,0.01550,0.01350,0.01050,0.0150,0.0020,...,0.00950,0.00400,0.01100,0.01000,0.00250,0.0010,0.0030,0.00100,0.000,3
131932,0.659662,0.0003,0.0009,0.0029,0.0058,0.00665,0.00680,0.00575,0.0066,0.0028,...,0.03670,0.02165,0.04940,0.06465,0.04640,0.0143,0.0213,0.01185,0.002,1
146867,0.734337,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,...,0.05000,0.00000,0.05000,0.10000,0.00000,0.1000,0.1000,0.00000,0.000,2


In [18]:
!pip install torch torchvision


In [19]:
import torch
import torch.nn as nn


In [20]:
# THE NETWORK

# Network class contains the layers needed for our network
# It inherits torch.nn.Module class to create the layers using torch.nn.Module class's methods.
class Network(torch.nn.Module):   
    
    # __init__() is called when an object of Network class is created.
    def __init__(self, number_of_features, number_of_classes): 
        
        super().__init__() 
 
        self.number_of_features = number_of_features # Store the number of features in the input dataset.
        self.number_of_classes = number_of_classes # Store the number of classes in the input dataset.

        kernel_size = 5
        padding = (kernel_size - 1) // 2
        
        # We have 2 convolutional layers in our network.
        # Below 2 statements defines the number of filtes (or out channels) to be used in our model.
        self.conv1_out_channels = 32
        # In the 2nd convolutional layer, the number of filters (or out channels) is doubled from the previous
        # convolutional layer.
        self.conv2_out_channels = self.conv1_out_channels * 2
        
        # Defining the convolutional layers of the model.
        # We have 2 convolutional layer in our model (self.conv1 and self.conv2) which are created as objects 
        # of the torch.nn.Conv1d class which are callable classes.
        # These objects are called in the forward method of this Network class by sending the input as argument.
        # The kernel size used in the convolution is 3
        # The padding size is kept 1 to keep the resulting output size same as that of the input size (same padding) 
        # For the 1st convolutional layer, the number of in channels is 1 as the number of channels in the input is 1.
        # The number of out channels is equal to the value of self.conv1_out_channels variable which is defined above.
        self.conv1 = torch.nn.Conv1d(in_channels=1, out_channels=self.conv1_out_channels, kernel_size=kernel_size, padding=padding)
        
        # For the 2nd convolutional layer, the number of in channels is equal to the out channels of the previous layer.
        # The number of out channels is equal to the value of self.conv2_out_cjhannels variable which is defined above.
        self.conv2 = torch.nn.Conv1d(in_channels=self.conv1_out_channels, out_channels=self.conv2_out_channels, kernel_size=kernel_size, padding=padding)

        self.pool = torch.nn.MaxPool1d(kernel_size=2, ceil_mode=True)
                     
        # We have 2 fully connected layers in pur model after the convolutinal layers
        # For the 1st fully connected layer, the number of input neurons will be equal to the number of elements
        # in the flattened output of the 2nd convolutional layer i.e. the number of out channels of the 2nd convolutional
        # layer multiplied by the number of features in the dataset (i.e. the size of the input)
        fc1_input_neurons = self.conv2_out_channels * (math.ceil(math.ceil(self.number_of_features / 2) / 2))
        
        # The number of output neurons is half of the input neurons.
        fc1_output_neurons = fc1_input_neurons // 2
      
        # Defining the fully connected layers.
        # The 2 fully connected layers of our model (self.fc1 and self.fc2) is created as objects of the 
        # torch.nn.Linear class which are callable classes and are called in the forward method of this network.
        # The 1st fully connected layer accepts the flattened output of the 2nd convolutional layer as input.
        self.fc1 = torch.nn.Linear(in_features=fc1_input_neurons, out_features=fc1_output_neurons) 
        
        # The 2nd fully connected layer accepts the output of the 1st fully connected layer as input.
        # The 2nd fully connected layer is the final layer of our model.
        # Its number of output neurons is equal to the number of classes in the dataset.
        self.fc2 = torch.nn.Linear(in_features=fc1_output_neurons, out_features=self.number_of_classes)
        
        # Defining ReLU activation function
        self.relu = torch.nn.ReLU() 
         
    # This method is called during the forward propagation
    # It accepts the input tensor and passes it through all the layers
    # and finally returns the resulting output tensor
    def forward(self, t):       
        
        conv1_output = self.conv1(t) # Feeding the input to the 1st convolutional layer.
        conv1_output = self.relu(conv1_output) 
        conv1_output = self.pool(conv1_output)

        conv2_output = self.conv2(conv1_output)  # Feeding the output of previous layer to the 2nd convolutional layer.
        conv2_output = self.relu(conv2_output) 
        conv2_output = self.pool(conv2_output)
        
        # Flattening the output of the 2nd convolutional layer in order to feed it to the 1st fully connected layer.
        flattened_output = conv2_output.reshape(-1, self.conv2_out_channels * (math.ceil(math.ceil(self.number_of_features / 2) / 2))) 
 
        # Feed the input to the fully connected layers
        fc1_output = self.fc1(flattened_output) # Feeding the flattened output to the 1st fully connected layer whose weights are now changed
        fc1_output = self.relu(fc1_output) # Applying relu activation on the output.
        fc2_output = self.fc2(fc1_output) # Feeding the output of previous layer to the 2nd fully connected layer.

        return fc2_output # return the output of the 2nd fully connected layer.

In [21]:
# We create a NumericalDataset class which inherits torch.utils.data.Dataset class 
# Its __init__() method accepts the features and classes of the dataset as arguments.
# It eases the process of feeding the dataset into our deep learning model.
class NumericalDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, y):
        self.x = x # x: the attributes
        self.y = y # y: the class labels
        
    def __len__(self): 
        return (len(self.x))
    
    def __getitem__(self, i):
        x = self.x[i, :] # Returns an 1D array.
        x = x.reshape(1, x.shape[0]) # Adding an extra dimension for channel [shape: (1, no_of_attributes)]
        x = x.double() 
        y = self.y[i].item() # Assuming y is a single number denoting the class
        y = int(y) # Assuming the class is denoted by integer
        return (x, y)

In [22]:
import torch
import time
import math 

# Assuming encode_dataframe, NumericalDataset, and Network classes are defined

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define the device


def get_data_loader(df, batch_size):
    np_data = df.values
    t_data = torch.tensor(np_data)
    x = t_data[:, :-1]
    y = t_data[:, -1]
    nd = NumericalDataset(x, y)
    data_loader = torch.utils.data.DataLoader(nd, batch_size=batch_size)
    return data_loader

total_epochs = 100
percentage = 10
batch_size = int(len(train_features) * (percentage / 100))
total_train_accuracy = 0
total_test_accuracy = 0

# print('Dataset:', dataset_name, dataset_number, '| Epoch:', total_epochs)
print()

start_time = time.time()

# Encoding train and test
# encoded_train, encoded_test = encode_dataframe(train_features, test_features)

# Storing the number of classes and features in the encoded dataframe
number_of_classes = int(max(encoded_train.iloc[:, -1]) + 1)
number_of_attributes = encoded_train.shape[1] - 1

train_loader = get_data_loader(encoded_train, batch_size=batch_size)
test_loader = get_data_loader(encoded_test, batch_size=batch_size)

# Creating a Network object for this fold
net = Network(number_of_attributes, number_of_classes).double().to(device)
optimizer = torch.optim.Adam(net.parameters())
criterion = torch.nn.CrossEntropyLoss()

# Training the model
for epoch in range(total_epochs):
    for batch in train_loader:
        train_x, train_y = batch
        train_x, train_y = train_x.to(device), train_y.to(device)
        shuffle_idx = torch.randperm(len(train_x))
        train_x = train_x[shuffle_idx]
        train_y = train_y[shuffle_idx]
        preds = net(train_x)
        loss = criterion(preds, train_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Testing the model
correct_predictions_train = 0
for batch in train_loader:
    train_x, train_y = batch
    train_x, train_y = train_x.to(device), train_y.to(device)
    preds = net(train_x)
    correct_predictions_train += preds.argmax(dim=1).eq(train_y).sum().item()
train_accuracy = round(correct_predictions_train / len(encoded_train) * 100, 2)

correct_predictions_test = 0
for batch in test_loader:
    test_x, test_y = batch
    test_x, test_y = test_x.to(device), test_y.to(device)
    preds = net(test_x)
    correct_predictions_test += preds.argmax(dim=1).eq(test_y).sum().item()
test_accuracy = round(correct_predictions_test / len(encoded_test) * 100, 2)

print('Train: %.2f Test: %.2f' % (train_accuracy, test_accuracy))

print('\nExecution Time:', round(time.time() - start_time, 2), 'seconds')


KeyboardInterrupt: 

In [ ]:
# This method accepts a DataFrame as argument and converts it into tensor.
def get_data_loader(df, batch_size):
        np_data = df.values # Fetching data as numpy array from the dataframe
        t_data = torch.tensor(np_data) # Converting the numpy array into tensor.
        x = t_data[:, :-1] # Attributes
        y = t_data[:, -1] # Classes
        nd = NumericalDataset(x, y) # creating the NumericalDataset object
        # data_loader (object of torch.utils.data.DataLoader class) will be used to load each row of the dataset
        # and feed it to our deep learning network.
        data_loader = torch.utils.data.DataLoader(nd, batch_size=batch_size) 
        return data_loader


total_epochs = 100 # The number of epochs with which we'll train the model.
percentage = 10 # Percentage of the train data to be taken in each batch
batch_size = int(len(train_list[0]) * (percentage / 100))
total_train_accuracy = 0 
total_test_accuracy = 0 # Add the accuracy of each fold. Divide it by 10 to get the final accuracy.
fold = 1

print('Dataset:', dataset_name, dataset_number, '| Epoch:', total_epochs)
print()

start_time = time.time()
 
for i in range(10):
    
    # Selecting train and test
    train_data, test_data = train_list[i], test_list[i]
    # Encoding train and test
    encoded_train, encoded_test = encode_dataframe(train_data, test_data)
    
    # Storing the number of classes and features in the encoded dataframe
    number_of_classes = int(max(encoded_train.iloc[:, -1]) + 1) 
    number_of_attributes = encoded_train.shape[1] - 1 

    train_loader = get_data_loader(encoded_train, batch_size=batch_size)
    test_loader = get_data_loader(encoded_test, batch_size=batch_size)
    
    # Creating a Network object for this fold (A new object is created for each fold)
    net = Network(number_of_attributes, number_of_classes).double().to(device) 
    
    # Defining the optimizer for our Network object. We apply learing rate of 0.001.
    optimizer = torch.optim.Adam(net.parameters())
    # Defining the loss calculating method for our Network object. We use cross entropy loss.
    criterion = torch.nn.CrossEntropyLoss()
 
    # Training our model.
    for epoch in range(total_epochs): 
        for batch in train_loader:
            
            # Get features and class labels in the v=batch and store them in train_x and train_y respectively
            train_x, train_y = batch
            train_x, train_y = train_x.to(device), train_y.to(device)

            # Shufle the batch
            shuffle_idx = torch.randperm(len(train_x)) # Get random indices
            train_x = train_x[shuffle_idx] # Arrange train_x and train_y with the random indices
            train_y = train_y[shuffle_idx]

            preds = net(train_x) # Forward propagation. Feed train_x as input into the model
            loss = criterion(preds, train_y) # Calculate loss
            optimizer.zero_grad()
            loss.backward() # Backpropagation. Calculate the Gradients
            optimizer.step() # Update the Weights  


    # Training of this fold is done 
    # Testing for this fold
    # Train accuracy
    correct_predictions_train = 0 
    for batch in train_loader:
        train_x, train_y = batch
        train_x, train_y = train_x.to(device), train_y.to(device)
        preds = net(train_x)
        correct_predictions_train += preds.argmax(dim=1).eq(train_y).sum().item()
    train_accuracy = round(correct_predictions_train / len(encoded_train) * 100, 2)
    # Test accuracy            
    correct_predictions_test = 0 
    for batch in test_loader:
        test_x, test_y = batch
        test_x, test_y = test_x.to(device), test_y.to(device)
        preds = net(test_x) 
        correct_predictions_test += preds.argmax(dim=1).eq(test_y).sum().item()
    test_accuracy = round(correct_predictions_test / len(encoded_test) * 100, 2)

    print('Train: %.2f Test: %.2f (Fold %d)' %(train_accuracy, test_accuracy, fold))
    total_train_accuracy += train_accuracy
    total_test_accuracy += test_accuracy
    fold += 1
    
print()
print('10CV result:')
final_train_accuracy = round(total_train_accuracy / 10, 2)
print('Train Accuracy:', final_train_accuracy)
final_test_accuracy = round(total_test_accuracy / 10, 2)
print('Test Accuracy:', final_test_accuracy)

print('\nExecution Time:', round(time.time() - start_time, 2), 'seconds')

In [ ]:
print(train.target)

In [ ]:
train

In [ ]:
import numpy as np

# Convert a list of lists to a NumPy array
X_train = np.array(train)
# Assuming X_train is your training data
# X_train should be a 2D NumPy array

# Initialize an empty array to store the scaled data
X_train_scaled = np.zeros_like(X_train)

# Iterate through each row of X_train
for i in range(X_train.shape[0]):
    row = X_train[i, :]  # Get the current row
    min_val = np.min(row)  # Calculate the minimum value in the row
    max_val = np.max(row)  # Calculate the maximum value in the row

    # Avoid division by zero
    if max_val != min_val:
        scaled_row = (row - min_val) / (max_val - min_val)
    else:
        scaled_row = np.zeros_like(row)  # If max and min are the same, set all values to 0

    X_train_scaled[i, :] = scaled_row  # Store the scaled row in the result array

# Now, X_train_scaled contains your normalized training data with each row scaled individually between 0 and 1




In [ ]:
print(X_train_scaled)

In [ ]:
# import numpy as np

# Assuming X_train_scaled is your scaled dataset
# Create an empty array for the formatted data
X_train_formatted = np.empty_like(X_train_scaled, dtype=np.float64)

# Iterate through each row and column of X_train_scaled
for i in range(X_train_scaled.shape[0]):
    for j in range(X_train_scaled.shape[1]):
        scaled_value = X_train_scaled[i, j]
        formatted_value = "{:.5f}".format(scaled_value)
        X_train_formatted[i, j] = float(formatted_value)

# Now, X_train_formatted contains your scaled dataset with all values formatted as decimals


In [ ]:
print(X_train_formatted)

In [ ]:
X_train_formatted.shape

In [ ]:
# import numpy as np

# # Example data (replace this with your actual data)
# train = np.array(train)

# # Function to perform Min-Max scaling
# def min_max_scaling(train):
#     # Calculate the minimum and maximum values for each feature
#     min_vals = np.min(train, axis=0)
#     max_vals = np.max(train, axis=0)
    
#     # Apply the Min-Max scaling formula
#     scaled_data = (train - min_vals) / (max_vals - min_vals)
    
#     return scaled_data, min_vals, max_vals

# # Apply Min-Max scaling
# scaled_data, min_vals, max_vals = min_max_scaling(train)

# # print("Original Data:\n", data)
# print("\nScaled Data (Min-Max scaled between 0 and 1):\n", scaled_data)
# print("\nMin Values for each feature:\n", min_vals)
# print("\nMax Values for each feature:\n", max_vals)

In [ ]:
                                                  #  PCA of train data

In [ ]:
# X_trainnew, X_val, y_trainnew, y_val = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# target_data=train['target']

In [ ]:
# target_data
# train=train.drop(columns=['target']) 

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# Assuming X_train_scaled is your scaled training data
# You can use X_train_formatted if you've converted it to human-readable format X_train_formatted

# Initialize the PCA model with the desired number of components
n_components = 33  # Adjust this value as needed
pca = PCA(n_components=n_components)

# Fit the PCA model to your scaled data
pca.fit(X_train_formatted)

# Transform the data to its principal components
X_train_pca = pca.transform(X_train_formatted)
# Create a DataFrame for the PCA results using common columns
pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i}' for i in range(1, 34)])
# Printing the PCA results
print(pca_df)

# Now, X_train_pca contains your training data reduced to the specified number of principal components
# You can use X_train_pca for classification

In [ ]:
# X_train_pca

In [ ]:
# Assuming you have a target_column, and pca_df is your DataFrame

# Add the target_column to pca_df
# pca_df['target'] = target_column


In [ ]:
print(pca_df)

In [ ]:
target_column

In [ ]:
X = pca_df  # Drop the target column to get the features
y = target_column

In [ ]:
y

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

X_trainnew, X_test, y_trainnew, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
y_trainnew

In [ ]:
X_trainnew

In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# # Assuming X and y are your feature matrix and target column
# # Replace 'X' and 'y' with your actual feature matrix and target column

# # Set the desired test size
# test_size = 0.1

# # Perform the train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

# # Print the sizes of the train and test sets
# print(f"Train Size: {len(X_train)}")
# print(f"Test Size: {len(X_test)}")


In [ ]:
# print(X_train)

In [ ]:
X_trainnew['target']=y_trainnew

In [ ]:
X_trainnew.columns

In [ ]:
X_trainnew

In [ ]:
# Check for NaN values in the resulting X_trainnew
print("NaN values in df_with_clusters:", X_trainnew.isna().sum().sum())

In [ ]:
    
# K-means algorithm
    
import pandas as pd
import numpy as np
import cudf
from cuml.cluster import KMeans as cuKMeans
from cudf import DataFrame as cudf_DataFrame

# Assuming df is your original DataFrame with PC1 to PC33 and the target column
# The target column should be named 'target'

# Extract numerical features for clustering X_trainnew
features = X_trainnew.columns[:-1]  # Exclude the target column
data_for_clustering = X_trainnew[features]

# Define the number of clusters
num_clusters = 100

# Initialize the KMeans model on GPU
kmeans = cuKMeans(n_clusters=num_clusters, random_state=0, n_init=1)

# Fit KMeans model
cluster_assignments_gpu = kmeans.fit_predict(data_for_clustering)

# Transfer cuML Series to cuDF DataFrame
cluster_assignments_cudf = cudf.Series(cluster_assignments_gpu)

# Convert cuDF Series to a NumPy array
cluster_assignments_cpu = cluster_assignments_cudf.to_pandas().values

# Add cluster assignments to the original DataFrame
df_with_clusters = pd.concat([X_trainnew, pd.DataFrame({'cluster': cluster_assignments_cpu})], axis=1)


# Check for NaN values in the resulting DataFrame
print("NaN values in df_with_clusters:", df_with_clusters.isna().sum().sum())


# Initialize a new DataFrame for the reduced dataset
reduced_data = pd.DataFrame(columns=df_with_clusters.columns)

# Randomly select 10,000 rows from the entire DataFrame with replacement
selected_rows = df_with_clusters.sample(n=100, replace=True, random_state=0)

# Check for NaN values in the selected rows
print("NaN values in selected_rows:", selected_rows.isna().sum().sum())

# Concatenate the selected rows to the reduced_data DataFrame
reduced_data = pd.concat([reduced_data, selected_rows], ignore_index=True)

  # Extract class labels from the "cluster" column
class_labels = reduced_data["cluster"].values

# Drop the "cluster" column from the reduced dataset
reduced_data = reduced_data.drop(columns=["cluster"])

# Check for NaN values in the reduced dataset
print("NaN values in reduced_data:", reduced_data.isna().sum().sum())
reduced_data.shape

In [ ]:
reduced_data.shape

In [ ]:
reduced_data

In [ ]:
X_trainnew

In [ ]:
reduced_data

In [ ]:
X_trainnew=X_trainnew.drop(columns=['target'])
# rain.drop(columns=['target']) 

In [ ]:
X_trainnew['target']=y_trainnew

In [ ]:
X_trainnew

In [ ]:
# X_trainnew=X_trainnew.drop(columns=['cluster_id'])


In [ ]:
# new_df.shape

In [ ]:
# new_df

In [ ]:
# Add this inside the loop before the condition
# print(f"Cluster {cluster_id}: Number of points = {len(each_cluster)}")


In [ ]:
# Assuming  target column is named 'target' and we want to use all 33 feature columns
X_train_1 = reduced_data.drop(columns=['target'])  # Extract all feature columns except 'target'
y_train_1 = reduced_data['target']  # Extract the 'target' column as labels


In [ ]:
X_train_1.shape

In [ ]:
X_train_1

In [ ]:
X_train_2 = X_train_1.drop(columns=['cluster_id'])  # Extract all feature columns except 'target'


In [ ]:
X_train_2

In [ ]:
y_train_1

In [ ]:
# print(reduced_data)


In [ ]:
X_train_1

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Handle missing values in X_train_1 by filling them with zeros
X_train_1_filled = X_train_1.fillna(0)

# Convert X_train_1 to a PyTorch tensor
X_tensor = torch.tensor(X_train_1_filled.values, dtype=torch.float32)

# Convert y_train_1 to strings to avoid mixed data types
y_train_1_str = y_train_1.astype(str)

# Encode the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train_1_str)

# Convert y_encoded to a PyTorch tensor
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Create a DataLoader
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Assuming number_of_features is the number of features in your input data
number_of_features = X_train_1.shape[1]

# Assuming number_of_classes is the number of unique classes in your target labels
number_of_classes = len(np.unique(y_train_1_str))

# Create an instance of the Network class
network = Network(number_of_features, 128, number_of_classes)  # Assuming hidden_size is 128

# Define your loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)

# Define validation dataset
X_val_1_filled = X_val_1.fillna(0)
X_val_tensor = torch.tensor(X_val_1_filled.values, dtype=torch.float32)
y_val_str = y_val_1.astype(str)
y_val_encoded = label_encoder.transform(y_val_str)
y_val_tensor = torch.tensor(y_val_encoded, dtype=torch.long)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Train your network
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = network(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy}")







In [ ]:
## Model Training
# 1:  (Random Forest)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

  # Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100,max_depth = 6 )  # You can adjust n_estimators
 # You can adjust n_estimators
  # You can adjust n_estimators

# Train the Random Forest Model
rf_classifier.fit(X_train_1, y_train_1)
# Step 4: Model Evaluation
# y_pred = rf_classifier.predict(pca_df_test)  # Predict on Test Data
y_pred = rf_classifier.predict(X_train_2)  # Predict on Test Data

accuracy = accuracy_score(y_train_1, y_pred,)
accuracy_percent = accuracy * 100
print(f"Accuracy: {accuracy_percent:.2f}%")  # Display accuracy in percentage format with two decimal places
# report = classification_report(y_true, y_pred,zero_division=1)

# print("Classification Report:\n", report)

In [ ]:
pca_df_test

In [ ]:
# Select the first 1000 rows
selected_rows = pca_df_test[:1000]

In [ ]:
selected_rows.shape

In [ ]:
y_selected_rows=y_true[:1000]

In [ ]:
y_selected_rows.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf_classifier = RandomForestClassifier(n_estimators=100,max_depth = 6 )  # You can adjust n_estimators
 # You can adjust n_estimators
  # You can adjust n_estimators

# Train the Random Forest Model
rf_classifier.fit(X_train_1, y_train_1)
# Step 4: Model Evaluation
# y_pred = rf_classifier.predict(pca_df_test)  # Predict on Test Data
y_pred = rf_classifier.predict(selected_rows)  # Predict on Test Data

accuracy = accuracy_score(y_selected_rows, y_pred)
accuracy_percent = accuracy * 100
print(f"Accuracy_2: {accuracy_percent:.2f}%")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf_classifier = RandomForestClassifier(n_estimators=1000,max_depth = 6 )  # You can adjust n_estimators
 # You can adjust n_estimators
  # You can adjust n_estimators

# Train the Random Forest Model
rf_classifier.fit(X_train_2, y_train_1)
# Step 4: Model Evaluation
y_pred = rf_classifier.predict(pca_df_test)  # Predict on Test Data
# y_pred = rf_classifier.predict(selected_rows)  # Predict on Test Data

accuracy = accuracy_score(y_true, y_pred)
accuracy_percent = accuracy * 100
print(f"Accuracy_2: {accuracy_percent:.2f}%")


In [ ]:
# with kmean_2
# # train accuracy check
# rf_classifier = RandomForestClassifier( n_estimators=20, max_depth=3,random_state=4)  # You can adjust n_estimators

# # Train the Random Forest Model
# rf_classifier.fit(X_train, y_train)

# # Step 4: Model Evaluation on Train Data
# y_train_pred = rf_classifier.predict(X_train)
# accuracy_train = accuracy_score(y_train, y_train_pred)
# accuracy_train_percent = accuracy_train * 100
# print(f"Train Accuracy: {accuracy_train_percent:.2f}%")  

In [ ]:
# 30/3/42--best accuracy==>76

In [ ]:
 ## 2. AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
 # Step 3: Initialize the base estimator (Decision Tree)
base_estimator = DecisionTreeClassifier()  # You can adjust max_depth
 # Step 4: Initialize the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50,random_state=32)
# You can adjust n_estimators (number of weak learners)max_depth=3,random_state=32
 # Step 5: Train the AdaBoost Model
adaboost_classifier.fit(X_train_2, y_train_1)
 # Step 6: Model Evaluation
y_pred = adaboost_classifier.predict(pca_df_test)
accuracy_test = accuracy_score(y_true, y_pred)
# report = classification_report(y_true, y_pred,zero_division=1)

accuracy_percent_test = accuracy_test * 100
print(f"Accuracy_test: {accuracy_percent_test:.2f}%")  # Display accuracy in percentage format with two decimal places

# print("Classification Report:\n", report)

In [ ]:
 ## 2. AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
 # Step 3: Initialize the base estimator (Decision Tree)
base_estimator = DecisionTreeClassifier()  # You can adjust max_depth
 # Step 4: Initialize the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50,random_state=32)
# You can adjust n_estimators (number of weak learners)max_depth=3,random_state=32
 # Step 5: Train the AdaBoost Model
adaboost_classifier.fit(X_train_2, y_train_1)
 # Step 6: Model Evaluation
y_pred = adaboost_classifier.predict(selected_rows)
accuracy_test = accuracy_score(y_selected_rows, y_pred)
# report = classification_report(y_true, y_pred,zero_division=1)

accuracy_percent_test = accuracy_test * 100
print(f"Accuracy_test: {accuracy_percent_test:.2f}%")  # Display accuracy in percentage format with two decimal places

# print("Classification Report:\n", report)

In [ ]:
### ## 2. AdaBoost Algorithm for train
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
 # Step 3: Initialize the base estimator (Decision Tree)
base_estimator = DecisionTreeClassifier(max_depth=3)  # You can adjust max_depth
 # Step 4: Initialize the AdaBoost Classifier 
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100, random_state=42)
# You can adjust n_estimators (number of weak learners)
 # Step 5: Train the AdaBoost Model
adaboost_classifier.fit(X_train_2, y_train_1)
 # Step 6: Model Evaluation
y_pred = adaboost_classifier.predict(X_train_2)
accuracy_3 = accuracy_score(y_train_1, y_pred)
# report = classification_report(y_true, y_pred,zero_division=1)

accuracy_percent_3 = accuracy_3 * 100
print(f"Accuracy_3: {accuracy_percent_3:.2f}%")  # Display accuracy in percentage format with two decimal places

# print("Classification Report:\n", report)

In [ ]:
# base_estimator = DecisionTreeClassifier(max_depth=3)  # You can adjust max_depth
#  # Step 4: Initialize the AdaBoost Classifier
# adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, random_state=42)===>>70%

In [ ]:
 # 3. ************  Naive Bayes Algorithm  ************
#     for train
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
# Create a pipeline with MinMaxScaler and Gaussian Naive Bayes
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Normalize/Scale features
    ('classifier', GaussianNB())  # Gaussian Naive Bayes classifier
])

# Train the model using the pipeline
pipeline.fit(X_train_2, y_train_1)

# Make predictions on the test set
y_pred = pipeline.predict(X_train_2)

# Evaluate the model
accuracy = accuracy_score(y_train_1, y_pred)

accuracy_percent = accuracy * 100
print(f"Accuracy_3: {accuracy_percent:.2f}%")

In [ ]:
 # 3. ************  Naive Bayes Algorithm  ************
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import MultinomialNB


from sklearn.model_selection import  GridSearchCV
 # Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())
#     ('classifier', MultinomialNB())  # Use Gaussian Naive Bayes classifier
])
 # Perform hyperparameter tuning using GridSearchCV
# param_grid = {
    # You can try different Naive Bayes variations and hyperparameters here
#     'classifier__var_smoothing': [1e-9, 1e-8, 1e-7],
# }
# Define a grid of hyperparameter values to search over
param_grid = {
    'classifier__var_smoothing': np.logspace(-9, 0, 10)  # Adjust the range of values as needed -9 0 10
}
 grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
#  grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid)

grid_search.fit(X_train_2, y_train_1)
 # Get the best estimator from the grid search
best_pipeline = grid_search.best_estimator_
 # Train the best pipeline on the training data
best_pipeline.fit(X_train_2, y_train_1)
 # Make predictions on the test data
y_pred = best_pipeline.predict(X_train_2)
 # Evaluate the model
accuracy = accuracy_score(y_train_1, y_pred)

accuracy_percent = accuracy * 100
print(f"Accuracy: {accuracy_percent:.2f}%") 


In [ ]:
## 4. SVM - Support Vector Machines code

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Support Vector Machine classifier
# You can adjust the kernel (linear, polynomial, radial basis function, etc.) and other hyperparameters

svm_classifier = SVC(kernel='linear', random_state=53,C=3.0)
# svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# Train the classifier on the training data
svm_classifier.fit(X_train_2, y_train_1)

# Make predictions on the test data
y_pre = svm_classifier.predict(X_train_2)

# Evaluate the model
accuracy = accuracy_score(y_train_1, y_pred)
accuracy_percent = accuracy * 100
print(f"Accuracy: {accuracy_percent:.2f}%") 
# Display classification report
# print("Classification Report:")
# print(classification_report(y_true, y_pred,zero_division=1))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Support Vector Machine classifier
# You can adjust the kernel (linear, polynomial, radial basis function, etc.) and other hyperparameters

# svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# Train the classifier on the training data
svm_classifier.fit(X_train_2, y_train_1)

# Make predictions on the test data
y_prediction = svm_classifier.predict(selected_rows)

# Evaluate the model
accuracy_6 = accuracy_score(y_selected_rows, y_prediction)
accuracy_percent_6 = accuracy_6 * 100
print(f"Accuracy: {accuracy_percent_6:.2f}%") 
# Display classification report
# print("Classification Report:")
# print(classification_report(y_true, y_pred,zero_division=1))

In [ ]:
# Save reduced_data to a CSV file
reduced_data.to_csv("reduced_data.csv", index=False)


In [ ]:
# Save reduced_data to a CSV file with a full path
reduced_data.to_csv("C:\\Users\\Rashmi\\Desktop\\reduced_data.csv", index=False)



In [ ]:
# Read the CSV file into a DataFrame
reloaded_data = pd.read_csv("reduced_data.csv")

# Now, reloaded_data contains the data from the CSV file


In [ ]:
file_path = r"C:\Users\Rashmi\Desktop\reduced_data\reduced_data"

try:
    reduced_data.to_csv(file_path, index=False)
    print(f"File saved successfully at: {file_path}")
except Exception as e:
    print(f"Error saving the file: {e}")


In [ ]:
print(reloaded_data)

In [ ]:
reduced_data

In [ ]:
# Random forest
# Step 3: Model Training (Random Forest)
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators = 1000, max_depth = 6, random_state=42)  # You can adjust n_estimators
# Train the Random Forest Model
rf_classifier.fit(X_train, y_train)
# Step 4: Model Evaluation
y_pred = rf_classifier.predict(X_test_pca)  # Predict on Test Data

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
#For Y_Train

import numpy as np

# Assuming cluster_centers_df contains the cluster labels from K-Means
cluster_mapping = {}  # Create a dictionary to map cluster labels to original target labels

for cluster_label, target_label in zip(cluster_centers_df, target_column_numpy):
    if cluster_label not in cluster_mapping:
        cluster_mapping[cluster_label] = []
    cluster_mapping[cluster_label].append(target_label)

# Now you have a mapping of cluster labels to target labels
# You can use this mapping to assign target labels to your clustered data
clustered_y_train = np.array([np.argmax(cluster_mapping[label]) for label in cluster_centers_df])


In [ ]:
# Load the sample submission CSV file
submission_df = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

# Extract the target labels for the first 100k rows (assuming your test data has 100k rows)
y_true = submission_df['target'][:100000]

In [ ]:
# # import cuml
# # import cudf
# # from cuml.cluster import KMeans
# import numpy as np
# import cudf
# import cuml
# from cuml.cluster import KMeans


# # # Assuming you have your training data in a NumPy array
# train_data_numpyy = np.array(result_array)  # Replace ... with your actual data

# # Convert the NumPy array to a cuDF DataFrame
# # train_data_cu = cudf.DataFrame(train_data_numpy)

# # Assuming you have a GPU-enabled Kaggle environment and data loaded into train_data

# # # Convert your train_data to a cuDF DataFrame
# # train_data_cu = cudf.DataFrame(train_data)

# # Define the number of clusters (K)
# n_clusters = 10000

# # Create and fit the K-means model
# kmeans = cuml.KMeans(n_clusters=n_clusters, n_init=100)
# kmeans.fit(train_data_numpyy)




# # Predict the cluster labels for the train data
# train_data_numpyy['cluster_labels'] = kmeans.labels_

# # Calculate the cluster centers (representative points)
# cluster_centers = kmeans.cluster_centers_

# # Create a DataFrame with cluster centers
# cluster_centers_df = cudf.DataFrame(cluster_centers)

# Now, cluster_centers_df contains K representative data points

In [ ]:
                          # Agorithms

In [ ]:
#Using SVM algorithm
import numpy as np
from sklearn.svm import SVC

In [ ]:

X_train = cluster_centers_df
y_train = clustered_y_train  

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())  # Use Gaussian Naive Bayes classifier
])
# Perform hyperparameter tuning using GridSearchCV
# param_grid = {
    # You can try different Naive Bayes variations and hyperparameters here
#     'classifier__var_smoothing': [1e-9, 1e-8, 1e-7],
# }
# Define a grid of hyperparameter values to search over
param_grid = {
    'classifier__var_smoothing': np.logspace(-9, 0, 10)  # Adjust the range of values as needed -9 0 10
}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
import pandas as pd

# Load the sample submission CSV file
submission_df = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

# Extract the target labels for the first 100k rows (assuming your test data has 100k rows)
y_true = submission_df['target'][:100000]

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have a trained SVM model named 'clf'

# Evaluate the model's performance on the test data
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
pca_df.head()

In [ ]:
pca_df.shape

In [ ]:
# import numpy as np
# from sklearn.cluster import KMeans

# # Assuming X_train_pca is the PCA-transformed training data
# # n_clusters is the number of clusters you want to create
# n_clusters = 3  # You can adjust this number based on your problem

# # Create a K-Means clustering model
# kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# # Fit the model to your PCA-transformed training data
# kmeans.fit(X_train_pca)

# # Get the cluster labels for each data point
# cluster_labels = kmeans.labels_

# You can now use cluster_labels for classification or analysis


In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the PCA-transformed data into training and test sets
# X_train_pca, X_test_pca, = train_test_split(X_projected, test_size=0.3333333, random_state=42)

# # Convert NumPy arrays to Pandas DataFrames
# X_train_pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i}' for i in range(1, 34)])
# X_test_pca_df = pd.DataFrame(data=X_test_pca, columns=[f'PC{i}' for i in range(1, 34)])



In [ ]:
X_train_pca.shape

In [ ]:
# X_test_pca_df

In [ ]:
X_test_pca.shape

In [ ]:
pip install --upgrade threadpoolctl


In [ ]:
print("Columns in pca_df:", pca_df.columns)
print("Columns in train:", train.columns)


In [ ]:
print("Index of pca_df:", pca_df.index)
print("Index of train:", train.index)


In [ ]:
print("Columns in train:", train.columns)



In [ ]:
print(reduced_df.columns)


In [ ]:
# Reducing rows using K-means Clustering
from sklearn.cluster import KMeans

# Extracting numeric columns from pca_df
numeric_columns = pca_df.select_dtypes(include=[np.number])
num_clusters = 10000

# Initialize KMeans with 'num_clusters' clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init=1)  #random_state is a parameter that controls the random number generator used by the algorithm.
                                                                   #n_init controls the number of times the algorithm will be run with different centroid initializations, and in your code, it's set to 1, meaning only one run.
cluster_assignments = kmeans.fit_predict(numeric_columns)
pca_df['cluster'] = cluster_assignments
# Assuming 'target_df' contains the target column and shares the same index as 'pca_df'
merged_df = pd.merge(pca_df, target_column, left_index=True, right_index=True)


averaged_records = []

# Loop through each unique cluster
for cluster_id in merged_df['cluster'].unique():
    cluster_records = merged_df[merged_df['cluster'] == cluster_id]
    
    # Counting occurrences of each class in the cluster
    class_counts = cluster_records['target'].value_counts()
    
    #class with the most occurrences (predominant class)
    predominant_class = class_counts.idxmax()
    
    # Select records of the predominant class and calculate their mean
    predominant_records = cluster_records[cluster_records['target'] == predominant_class]
    averaged_record = predominant_records.drop(columns=['cluster', 'target']).mean()
    
    # Assigning the predominant class to the averaged record
    averaged_record['target'] = predominant_class
    
    # Appending the averaged record to the list
    averaged_records.append(averaged_record)

# Create the 'reduced_df' DataFrame from the list of averaged records
reduced_df = pd.DataFrame(averaged_records)

# 'reduced_df'contains the reduced and averaged records with the predominant class


In [ ]:
reduced_df.shape

In [ ]:
reduced_df.head()

In [ ]:
# Assuming your target column is named 'target' and you want to use all 33 feature columns
X_train = reduced_df.drop(columns=['target'])  # Extract all feature columns except 'target'
y_train = reduced_df['target']  # Extract the 'target' column as labels


In [ ]:
from sklearn.svm import SVC

# Assuming you have your training data X_train and y_train
# clf = SVC(kernel='poly')  # Example with a linear kernel
# clf.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = clf.predict(X_test_pca)


In [ ]:
import pandas as pd

# Load the sample submission CSV file
submission_df = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

# Extract the target labels for the first 100k rows (assuming your test data has 100k rows)
y_true = submission_df['target'][:100000]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have a trained SVM model named 'clf'

# Evaluate the model's performance on the test data
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)



In [ ]:
# y=reduced_df['target']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=1000,max_depth=6, random_state=42)
# rf_classifier = RandomForestClassifier(
#     n_estimators=1000,
#     max_depth=6,
#     min_samples_split=5,
#     class_weight="balanced",
#     random_state=42
# )


In [ ]:
# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = rf_classifier.predict(X_test_pca)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")



In [ ]:
# You can also print a detailed classification report
report = classification_report(y_true, y_pred )
print("Classification Report:\n", report)

In [ ]:
# print("Accuracy:", accuracy)

In [ ]:
# Naive Bayes classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV


In [ ]:
# Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())  # Use Gaussian Naive Bayes classifier
])

In [ ]:
# Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())  # Use Gaussian Naive Bayes classifier
])
# Perform hyperparameter tuning using GridSearchCV
# param_grid = {
    # You can try different Naive Bayes variations and hyperparameters here
#     'classifier__var_smoothing': [1e-9, 1e-8, 1e-7],
# }
# Define a grid of hyperparameter values to search over
param_grid = {
    'classifier__var_smoothing': np.logspace(-9, 0, 10)  # Adjust the range of values as needed -9 0 10
}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator from the grid search
best_pipeline = grid_search.best_estimator_

In [ ]:
# Train the best pipeline on the training data
best_pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = best_pipeline.predict(X_test_pca)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print( accuracy)

In [ ]:
# Initialize a Gaussian Naive Bayes classifier
# nb_classifier = GaussianNB()

In [ ]:
# Train the classifier
# nb_classifier.fit(X_train, y_train)


In [ ]:
# Train the pipeline (scaling and classification)
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = pipeline.predict(X_test_pca)

In [ ]:
# C4.5 Decision Tree classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
# Create a C4.5 Decision Tree classifier
# c45_classifier = DecisionTreeClassifier(criterion='entropy',splitter='best')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create a DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()

In [ ]:
# Train the classifier on the training data
dt_classifier.fit(X_train, y_train)


In [ ]:
# Make predictions on the test data
y_pred = dt_classifier.predict(X_test_pca)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

In [ ]:
# Step 5: Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)